# MLOps Workshop: Modell-Engineering und Training

## Einführung
In diesem Notebook werden wir uns mit dem Modell-Engineering und Training für unser Customer Churn Prediction Projekt beschäftigen. Wir werden MLflow für das Experiment-Tracking verwenden und verschiedene Modelle evaluieren.

## Lernziele
Nach Abschluss dieses Notebooks werden Sie:
- Verstehen, wie man MLflow für Experiment-Tracking einsetzt
- Verschiedene Modelle mit unterschiedlichen Hyperparametern trainieren können
- Best Practices für Modell-Evaluierung kennen
- Wissen, wie man Modelle im MLflow Model Registry registriert

## 1. Setup und Daten laden


In [4]:
# Benötigte Bibliotheken importieren
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# MLflow Tracking lokal einrichten (ohne Server)
mlflow.set_tracking_uri("file:./mlruns")


In [45]:
# Daten laden und vorbereiten
processed_data = pd.read_csv('../data/processed/telco_customer_churn_processed.csv')

# Numerische Spalten standardisieren
numeric_columns = ['tenure', 'MonthlyCharges', 'TotalCharges']
scaler = StandardScaler()
processed_data[numeric_columns] = scaler.fit_transform(processed_data[numeric_columns])

# One-Hot-Encoding für kategorische Variablen
categorical_columns = processed_data.select_dtypes(include=['object']).columns
categorical_columns = [col for col in categorical_columns if col not in ['Churn', 'customerID']]

# One-Hot-Encoding anwenden
X = pd.get_dummies(processed_data.drop(['Churn', 'customerID'], axis=1), columns=categorical_columns)
y = processed_data['Churn']

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 2. MLflow Experiment erstellen

### Aufgabe 1
Erstellen Sie ein neues MLflow Experiment für unser Churn-Vorhersage-Projekt. Setzen Sie einen aussagekräftigen Namen und eine Beschreibung.

<details>
<summary>👉 Lösung anzeigen</summary>

```python
# MLflow Experiment erstellen
experiment_name = "customer_churn_prediction"
experiment = mlflow.set_experiment(experiment_name)

# Experiment-Details ausgeben
print(f"Experiment Name: {experiment_name}")
print(f"Experiment ID: {experiment.experiment_id}")
```
</details>

## 3. Modell-Training mit MLflow Tracking

### Aufgabe 2
Implementieren Sie eine Funktion, die ein Modell trainiert und die wichtigsten Metriken mit MLflow trackt.

<details>
<summary>👉 Lösung anzeigen</summary>

```python
def train_and_evaluate_model(model, X_train, X_test, y_train, y_test, model_name, params=None):
    """
    Trainiert ein Modell und trackt die Ergebnisse mit MLflow
    """
    with mlflow.start_run() as run:
        # Parameter loggen
        if params:
            mlflow.log_params(params)
        
        # Modell trainieren
        model.fit(X_train, y_train)
        
        # Vorhersagen
        y_pred = model.predict(X_test)
        
        # Metriken berechnen
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, pos_label=1)
        recall = recall_score(y_test, y_pred, pos_label=1)
        f1 = f1_score(y_test, y_pred, pos_label=1)
        
        # Metriken loggen
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        
        # Modell loggen
        mlflow.sklearn.log_model(model, "model")
        
        # Feature Importance Plot erstellen und loggen
        if hasattr(model, 'feature_importances_'):
            plt.figure(figsize=(10, 6))
            feature_importance = pd.DataFrame({
                'feature': X_train.columns,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            sns.barplot(data=feature_importance, x='importance', y='feature')
            plt.title(f'Feature Importance - {model_name}')
            plt.tight_layout()
            
            # Plot als Artefakt speichern
            plt.savefig('feature_importance.png')
            mlflow.log_artifact('feature_importance.png')
            plt.close()
            
        return run.info.run_id, {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        }
```
</details>


### Aufgabe 3
Trainieren Sie verschiedene Modelle mit unterschiedlichen Hyperparametern und tracken Sie die Ergebnisse.

<details>
<summary>👉 Lösung anzeigen</summary>

```python
# Random Forest mit verschiedenen Hyperparametern
rf_params_list = [
    {
        'n_estimators': 100,
        'max_depth': 10,
        'min_samples_split': 2,
        'random_state': 42
    },
    {
        'n_estimators': 200,
        'max_depth': 15,
        'min_samples_split': 5,
        'random_state': 42
    },
    {
        'n_estimators': 300,
        'max_depth': None,
        'min_samples_split': 10,
        'random_state': 42
    }
]

results = []
for params in rf_params_list:
    model = RandomForestClassifier(**params)
    run_id, metrics = train_and_evaluate_model(
        model, X_train, X_test, y_train, y_test,
        model_name="RandomForest",
        params=params
    )
    results.append({
        'run_id': run_id,
        'params': params,
        'metrics': metrics
    })

# Ergebnisse als DataFrame anzeigen
results_df = pd.DataFrame([
    {
        'run_id': r['run_id'],
        **r['params'],
        **r['metrics']
    } for r in results
])
print("\nModell-Vergleich:")
print(results_df)
```
</details>


## 4. Modell im MLflow Model Registry registrieren

### Aufgabe 4
Wählen Sie das beste Modell aus und registrieren Sie es im MLflow Model Registry.

<details>
<summary>👉 Lösung anzeigen</summary>

```python
# Bestes Modell anhand des F1-Scores auswählen
best_run = max(results, key=lambda x: x['metrics']['f1_score'])
best_run_id = best_run['run_id']

# Modell im Registry registrieren
model_name = "customer_churn_predictor"
model_version = mlflow.register_model(
    f"runs:/{best_run_id}/model",
    model_name
)

print(f"Model '{model_name}' wurde als Version {model_version.version} registriert")
```
</details>


## 5. Modell-Staging und Deployment vorbereiten

### Aufgabe 5
Setzen Sie das registrierte Modell auf "Staging" und dokumentieren Sie die Modell-Details.

<details>
<summary>👉 Lösung anzeigen</summary>

```python
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Modell auf "Staging" setzen
client.transition_model_version_stage(
    name=model_name,
    version=model_version.version,
    stage="Staging",
    archive_existing_versions=True
)

# Modell-Details dokumentieren
client.update_model_version(
    name=model_name,
    version=model_version.version,
    description=f"""
    Customer Churn Prediction Model v{model_version.version}
    
    - Modelltyp: Random Forest
    - Training durchgeführt am: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}
    - Beste Parameter: {best_run['params']}
    - Performance Metriken:
        - Accuracy: {best_run['metrics']['accuracy']:.4f}
        - Precision: {best_run['metrics']['precision']:.4f}
        - Recall: {best_run['metrics']['recall']:.4f}
        - F1-Score: {best_run['metrics']['f1_score']:.4f}
    """
)
```
</details>



## Hausaufgaben und weiterführende Übungen
1. Experimentieren Sie mit weiteren Modelltypen (z.B. XGBoost, LightGBM)
2. Implementieren Sie eine Cross-Validation in den Training-Prozess
3. Erweitern Sie das Feature Importance Plot um weitere Visualisierungen
4. Fügen Sie Custom Tags zu den MLflow Runs hinzu
5. Implementieren Sie eine automatische Modellversionierung basierend auf Performance-Metriken

## Nächste Schritte
- Fahren Sie mit dem Deployment-Notebook fort
- Lernen Sie, wie man das Modell mit FastAPI bereitstellt
- Implementieren Sie Monitoring für das deployte Modell

## Hilfreiche MLflow-Befehle
```bash
# MLflow UI starten
mlflow ui

# Experiments auflisten
mlflow experiments list

# Run Details anzeigen
mlflow runs describe <run-id>

# Modell laden
mlflow models load runs:/<run-id>/model
```